# Predicting NBA Salaries

### 1) Set up the enviornment.

In [20]:
import sqlalchemy
import numpy as np 
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

#### Read the Data

In [21]:
#create the engine to read mySQL from pandas. To do this, the engine string must be as follows: 
#'mysql+pymysql://(user):(pw)@(server)/(schema)'. I won't show this due to sensitive info
engine = sqlalchemy.create_engine('mysql+pymysql://(user):(pw)@(server)/(schema)')

In [22]:
raw_data = pd.read_sql('player_info',engine)
train_data =  pd.read_sql('player_info',engine)

raw_data.head()

,games_played,Player,season,min,fgm,fga,fg3m,fg3a,ftm,fta,...,blk,turnover,pf,pts,fg_pct,fg3_pct,ft_pct,Unnamed: 0,Tm,season17_18
0,75,Alex Abrines,2017,15:07,1.53,3.88,1.12,2.95,0.52,0.61,...,0.11,0.33,1.65,4.71,0.395,0.380,0.848,185,OKC,5725000.0
1,76,Steven Adams,2017,32:43,5.89,9.37,0.00,0.03,2.11,3.76,...,1.03,1.68,2.83,13.89,0.629,0.000,0.559,32,OKC,22471910.0
2,69,Bam Adebayo,2017,19:49,2.52,4.93,0.00,0.10,1.87,2.59,...,0.59,0.96,2.00,6.91,0.512,0.000,0.721,281,MIA,2490360.0
3,75,LaMarcus Aldridge,2017,33:26,9.16,17.96,0.36,1.23,4.45,5.32,...,1.20,1.48,2.15,23.13,0.510,0.293,0.837,36,SAS,21461010.0
4,69,Al-Farouq Aminu,2017,30:01,3.33,8.43,1.81,4.91,0.86,1.16,...,0.58,1.14,1.97,9.33,0.395,0.369,0.738,154,POR,7319035.0


### 2) Clean the data

Some columns in this data need to be clarified. Unnamed: 0 is an index and must be removed. Since team is categorical data, we must choose between label encoding and dummy variables. Since the teams aren't in any order w.r.t salary, we must use dummy variables

In [23]:
#rename and drop
train_data.rename(columns={'season17_18': 'salary','Tm': 'team'}, inplace=True)
train_data.drop(['Unnamed: 0'], axis = 1,inplace=True)

#dummy variables for categorical data
dv = pd.get_dummies(train_data['team'])
train_data.drop(['team'], axis = 1,inplace=True)
train_data = pd.concat([train_data, dv],axis=1) #appends df based on matching indexes
train_data.head()

,games_played,Player,season,min,fgm,fga,fg3m,fg3a,ftm,fta,...,OKC,ORL,PHI,PHO,POR,SAC,SAS,TOR,UTA,WAS
0,75,Alex Abrines,2017,15:07,1.53,3.88,1.12,2.95,0.52,0.61,...,1,0,0,0,0,0,0,0,0,0
1,76,Steven Adams,2017,32:43,5.89,9.37,0.00,0.03,2.11,3.76,...,1,0,0,0,0,0,0,0,0,0
2,69,Bam Adebayo,2017,19:49,2.52,4.93,0.00,0.10,1.87,2.59,...,0,0,0,0,0,0,0,0,0,0
3,75,LaMarcus Aldridge,2017,33:26,9.16,17.96,0.36,1.23,4.45,5.32,...,0,0,0,0,0,0,1,0,0,0
4,69,Al-Farouq Aminu,2017,30:01,3.33,8.43,1.81,4.91,0.86,1.16,...,0,0,0,0,1,0,0,0,0,0


Check for nulls

In [24]:
#having two any's will only print if there is a null or not
train_data.isnull().any().any()

False

Check for duplicated values

In [25]:
train_data.duplicated(subset=['Player']).any()

True

There are duplicated players, so we must drop them

In [26]:
train_data.drop_duplicates(subset=['Player'],inplace=True)

#we must alse drop from raw data to avoid shape errors
raw_data.drop_duplicates(subset=['Player'],inplace=True)

### 3) Feature Engineering

In [27]:
#Create correlation matrix
corr = train_data.corr()
display(corr['salary'])

games_played    0.135526
season               NaN
fgm             0.559731
fga             0.538981
fg3m            0.328742
fg3a            0.339758
ftm             0.553186
fta             0.553808
oreb            0.271886
dreb            0.449326
reb             0.420418
ast             0.429383
stl             0.336814
blk             0.289622
turnover        0.493863
pf              0.302592
pts             0.575679
fg_pct          0.113973
fg3_pct         0.035967
ft_pct          0.099389
salary          1.000000
ATL            -0.038897
BOS            -0.004827
BRK            -0.014559
CHI            -0.080581
CHO             0.025447
CLE             0.075597
DAL            -0.034338
DEN            -0.007852
DET             0.056488
GSW             0.067882
HOU             0.048378
IND            -0.051179
LAC             0.027547
LAL             0.008045
MEM             0.021748
MIA             0.007767
MIL             0.021374
MIN             0.039611
NOP             0.067607


In [28]:
#drop indexes with no correlation, since they are irrelevant data
for x in list(train_data.columns):
    if x not in list(corr.columns):
        train_data.drop([x], axis = 1,inplace=True)

print("Remaining columns:",list(train_data.columns))

Remaining columns: ['games_played', 'season', 'fgm', 'fga', 'fg3m', 'fg3a', 'ftm', 'fta', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'turnover', 'pf', 'pts', 'fg_pct', 'fg3_pct', 'ft_pct', 'salary', 'ATL', 'BOS', 'BRK', 'CHI', 'CHO', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']


In [29]:
#season needs to be dropped since the correlation is NaN
train_data.drop(['season'], axis = 1,inplace=True)
train_data.head()

,games_played,fgm,fga,fg3m,fg3a,ftm,fta,oreb,dreb,reb,...,OKC,ORL,PHI,PHO,POR,SAC,SAS,TOR,UTA,WAS
0,75,1.53,3.88,1.12,2.95,0.52,0.61,0.35,1.17,1.52,...,1,0,0,0,0,0,0,0,0,0
1,76,5.89,9.37,0.00,0.03,2.11,3.76,5.05,3.96,9.01,...,1,0,0,0,0,0,0,0,0,0
2,69,2.52,4.93,0.00,0.10,1.87,2.59,1.71,3.81,5.52,...,0,0,0,0,0,0,0,0,0,0
3,75,9.16,17.96,0.36,1.23,4.45,5.32,3.28,5.19,8.47,...,0,0,0,0,0,0,1,0,0,0
4,69,3.33,8.43,1.81,4.91,0.86,1.16,1.41,6.20,7.61,...,0,0,0,0,1,0,0,0,0,0


### 4) Model

Since most of the variables fails to fit a linear relationship, the criteria for multiple linear regression isn't met. XGBoost is the appropriate model.

In [30]:
#create x and y variables
y = train_data['salary']
x = train_data.drop(['salary'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1)

#create XGBoost model and print MAE
model = xgb.XGBClassifier(learning_rate=0.05).fit(x_train, y_train)
pred_test = model.predict(x_test)
val_mae = mean_absolute_error(pred_test, y_test)
print(val_mae)

5205773.044117647


### 5) Present Results

For this project, we wanted to predict the salaries of NBA players. We need to output the name of the player, his current salary, his predicted salary, the difference, and whether he's overpaid or underpaid.

In [31]:
pred = model.predict(x)

In [34]:
#create a function for overpaid/underpaid
def over_under(df):
    copy = df.copy()
    for x in list(copy.index): #list(copy.index) will account for deleted rows
        if copy.loc[x,'index'] < 0: 
            copy.loc[x,'index'] = 'Underpaid'
        elif copy.loc[x,'index'] == 0:
            copy.loc[x,'index'] = 'Balanced'
        else:
            copy.loc[x,'index'] = 'Overpaid'
                   
    return copy

#create dataframes for difference and overpaid/underpaid
diff= pd.DataFrame({'index' : raw_data['season17_18']-pred})
overunder = over_under(diff)

#output results
output = pd.DataFrame({'Player': raw_data['Player'], 'Salary': raw_data['season17_18'], 'Predicted Salary' : pred, 'Difference' : diff['index'], 'Over/Under' : overunder['index']})
output.to_csv('salary_pred.csv', index=False)
print("Your submission was successfully saved!")
output

Your submission was successfully saved!


,Player,Salary,Predicted Salary,Difference,Over/Under
0,Alex Abrines,5725000.0,815615.0,4909385.0,Overpaid
1,Steven Adams,22471910.0,22471910.0,0.0,Balanced
2,Bam Adebayo,2490360.0,17000000.0,-14509640.0,Underpaid
3,LaMarcus Aldridge,21461010.0,23775506.0,-2314496.0,Underpaid
4,Al-Farouq Aminu,7319035.0,16400000.0,-9080965.0,Underpaid
...,...,...,...,...,...
349,Delon Wright,1645200.0,1471382.0,173818.0,Overpaid
350,Guerschon Yabusele,2247480.0,815615.0,1431865.0,Overpaid
351,Thaddeus Young,14796348.0,8000000.0,6796348.0,Overpaid
352,Cody Zeller,12584270.0,12500000.0,84270.0,Overpaid


#### For the love of data, lets find the most overpaid and underpaid players in the 2017-2018 season

In [33]:
i = output['Difference'].argmax()
print("The most overpaid player is :",output.loc[i,'Player'])
i = output['Difference'].argmin()
print("The most underpaid player is :",output.loc[i,'Player'])

The most overpaid player is : Joe Harris
The most underpaid player is : DeAndre Jordan
